In [44]:
import pandas as pd
import re
import string
from datetime import datetime
import plotly.express as px

In [46]:
air_jan23 = pd.read_csv("./Data/air_jan23.csv")
airport = pd.read_csv("./Data/L_AIRPORT.csv")
state_fips = pd.read_csv("./Data/L_STATE_FIPS.csv")
wac = pd.read_csv("./Data/L_WORLD_AREA_CODES.csv")

In [47]:
air_jan23.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,...,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2023,1,1,1,7,1/1/2023 12:00:00 AM,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,135.0,113.0,1.0,692.0,3,NaN,NaN,NaN,NaN,NaN
1,2023,1,1,1,7,1/1/2023 12:00:00 AM,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,102.0,78.0,1.0,481.0,2,NaN,NaN,NaN,NaN,NaN
2,2023,1,1,1,7,1/1/2023 12:00:00 AM,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,105.0,81.0,1.0,481.0,2,NaN,NaN,NaN,NaN,NaN
3,2023,1,1,1,7,1/1/2023 12:00:00 AM,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,96.0,79.0,1.0,481.0,2,NaN,NaN,NaN,NaN,NaN
4,2023,1,1,1,7,1/1/2023 12:00:00 AM,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,172.0,151.0,1.0,1041.0,5,256.0,0.0,1.0,0.0,0.0


In [48]:
#airport.head()

In [49]:
#state_fips.head()

In [50]:
wac.head()

,Code,Description
0,1,Alaska
1,2,Hawaii
2,3,Puerto Rico
3,4,U.S. Virgin Islands
4,5,U.S. Pacific Trust Territories and Possessions


## 1. Prepare the table
**Left join the WAC codes with the main table and change the date to `datetime` object**

In [51]:
wac_dest = wac

In [52]:
#airport = airport.rename(columns={'Code':'ORIGIN'})
wac = wac.rename(columns={'Code':'ORIGIN_WAC', 'Description':'ORIGIN_LOC'})
wac_dest = wac_dest.rename(columns={'Code':'DEST_WAC', 'Description':'DEST_LOC'})

In [53]:
air_jan23 = air_jan23.merge(wac, how="left", on="ORIGIN_WAC")
air_jan23 = air_jan23.merge(wac_dest, how="left", on="DEST_WAC")
air_jan23['FL_DATE'] = air_jan23['FL_DATE'].str.extract(r'(^\S+)')
air_jan23['FL_DATE'] = air_jan23['FL_DATE'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').date())

In [54]:
air_jan23.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,...,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,ORIGIN_LOC,DEST_LOC
0,2023,1,1,1,7,2023-01-01,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,1.0,692.0,3,NaN,NaN,NaN,NaN,NaN,Pennsylvania,Georgia
1,2023,1,1,1,7,2023-01-01,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,1.0,481.0,2,NaN,NaN,NaN,NaN,NaN,Pennsylvania,North Carolina
2,2023,1,1,1,7,2023-01-01,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,1.0,481.0,2,NaN,NaN,NaN,NaN,NaN,Pennsylvania,North Carolina
3,2023,1,1,1,7,2023-01-01,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,1.0,481.0,2,NaN,NaN,NaN,NaN,NaN,Pennsylvania,North Carolina
4,2023,1,1,1,7,2023-01-01,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,1.0,1041.0,5,256.0,0.0,1.0,0.0,0.0,Pennsylvania,Florida


In [55]:
# Checkpoint
#air_jan23_1 = air_jan23

In [56]:
original_length = len(air_jan23)
original_length

573877

## Cancelled Flights

In [57]:
air_jan23_cancel = air_jan23[air_jan23['CANCELLED'] == 1]

In [58]:
len(air_jan23_cancel)/len(air_jan23)

0.01922363154473868

Around 2% of flights were cancelled

## Delayed Flights

### Cleaning the Data

In [59]:
air_jan23_flew = air_jan23[air_jan23['CANCELLED'] == 0]

In [60]:
air_jan23_flew['ARR_DELAY'].isna().sum()

1480

In [61]:
air_jan23_flew['ARR_TIME'].isna().sum()

243

In [62]:
air_jan23_flew['CRS_ARR_TIME'].isna().sum()

0

There are 1480 NaN values for arrival delay amongst flights that flown. <br>
There are scheduled arrival times listed for every flight, but 243 missing arrival times. Therefore, we need to remove anyrows that lack both `ARR_TIME` and `ARR_DELAY` <br>

The original number of observations was 573877, but after removing the NA's (243 obs), we get 561365 rows.

In [63]:
air_jan23 = air_jan23[air_jan23['ARR_TIME'].notna() & air_jan23['ARR_DELAY'].notna()]

In [64]:
len(air_jan23)

561365

It seems like we don't have any other NA values for our arrival times,

In [65]:
air_jan23['ARR_DELAY'].isna().sum()

0

### Proportion of Delayed Flights

In [66]:
delayed_flights = air_jan23[air_jan23['ARR_DELAY'] > 0]
len(delayed_flights)

215103

Approximately **38%** of the flown (not cancelled) flights were delayed

In [67]:
len(delayed_flights)/len(air_jan23)

0.383178502400399

### Delay (in min) and Outlier Examinations

In [68]:
delayed_flights['ARR_DELAY'].describe()

count    215103.000000
mean         43.395699
std          81.559332
min           1.000000
25%           7.000000
50%          19.000000
75%          48.000000
max        3063.000000
Name: ARR_DELAY, dtype: float64

In [69]:
delay_reason = delayed_flights[delayed_flights.columns[-6:-2]].fillna(0)
delay_reason

,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
3,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,21.0,7.0,0.0,0.0
9,0.0,0.0,0.0,0.0
...,...,...,...,...
573863,10.0,20.0,0.0,11.0
573864,54.0,0.0,0.0,20.0
573865,0.0,0.0,0.0,0.0
573867,0.0,0.0,0.0,0.0


Most of the reasons for overall delays are due to the same flight arriving at a previous airport late (domino effect).

In [76]:
fig = px.pie(values = delay_reason.sum(), names=delay_reason.sum().index, title='Overall Delay Reasons Distribution')
fig.show()

In [77]:
#create a function to find outliers using IQR
def find_outliers_IQR(df):
    q1 = df.quantile(0.25)
    q3 =df.quantile(0.75)
    IQR=q3-q1
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return outliers

In [78]:
delayed_flights['ARR_DELAY']

3           2.0
4         257.0
7           3.0
8          28.0
9          17.0
          ...  
573863     41.0
573864     74.0
573865     68.0
573867     40.0
573876      5.0
Name: ARR_DELAY, Length: 215103, dtype: float64

In [79]:
outliers = find_outliers_IQR(delayed_flights['ARR_DELAY'])
print('number of outliers: '+ str(len(outliers)))
print('max outlier value: '+ str(outliers.max()))
print('min outlier value: '+ str(outliers.min()))
outliers=outliers.reset_index()
# table of delayed flights with just the outliers
delayed_outlier = delayed_flights[delayed_flights.index.isin(outliers['index'])]
delayed_outlier

number of outliers: 20188
max outlier value: 3063.0
min outlier value: 110.0


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,...,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,ORIGIN_LOC,DEST_LOC
4,2023,1,1,1,7,2023-01-01,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,...,1.0,1041.0,5,256.0,0.0,1.0,0.0,0.0,Pennsylvania,Florida
40,2023,1,1,1,7,2023-01-01,ABQ,"Albuquerque, NM",NM,35,...,1.0,744.0,3,20.0,0.0,17.0,0.0,225.0,New Mexico,Texas
65,2023,1,1,1,7,2023-01-01,ABR,"Aberdeen, SD",SD,46,...,1.0,257.0,2,124.0,0.0,0.0,0.0,0.0,South Dakota,Minnesota
69,2023,1,1,1,7,2023-01-01,ACT,"Waco, TX",TX,48,...,1.0,89.0,1,244.0,0.0,0.0,0.0,0.0,Texas,Texas
82,2023,1,1,1,7,2023-01-01,AEX,"Alexandria, LA",LA,22,...,1.0,500.0,3,0.0,83.0,2.0,0.0,342.0,Louisiana,Georgia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573850,2023,1,1,31,2,2023-01-31,XNA,"Fayetteville, AR",AR,5,...,1.0,280.0,2,0.0,317.0,54.0,0.0,0.0,Arkansas,Texas
573853,2023,1,1,31,2,2023-01-31,XNA,"Fayetteville, AR",AR,5,...,1.0,438.0,2,0.0,254.0,12.0,0.0,74.0,Arkansas,Texas
573855,2023,1,1,31,2,2023-01-31,XNA,"Fayetteville, AR",AR,5,...,1.0,1371.0,6,0.0,219.0,0.0,0.0,0.0,Arkansas,California
573857,2023,1,1,31,2,2023-01-31,XNA,"Fayetteville, AR",AR,5,...,1.0,1147.0,5,0.0,7.0,92.0,0.0,94.0,Arkansas,New York


In [80]:
delayed_outlier['ARR_DELAY'].describe()

count    20188.000000
mean       216.859025
std        177.605006
min        110.000000
25%        131.000000
50%        163.000000
75%        223.000000
max       3063.000000
Name: ARR_DELAY, dtype: float64

In [81]:
delay_reason_out = delayed_outlier[delayed_outlier.columns[-6:-2]].fillna(0)
delay_reason_out

,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
4,0.0,1.0,0.0,0.0
40,0.0,17.0,0.0,225.0
65,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0
82,83.0,2.0,0.0,342.0
...,...,...,...,...
573850,317.0,54.0,0.0,0.0
573853,254.0,12.0,0.0,74.0
573855,219.0,0.0,0.0,0.0
573857,7.0,92.0,0.0,94.0


For more extreme delays, there is a significantly higher portion of delays due to late aircraft, but the proportions of NAS delays and weather delays are a bit closer to each other than overall.

In [82]:
fig = px.pie(values = delay_reason_out.sum(), names=delay_reason_out.sum().index, title='Extreme Delay Reasons Distribution')
fig.show()

In [83]:
# table of delayed flights excluding outliers
delayed_inrange = delayed_flights[~delayed_flights.index.isin(outliers['index'])] # len 194915
delayed_inrange['ARR_DELAY'].describe()
#len(delayed_inrange)  

count    194915.000000
mean         25.429521
std          25.193077
min           1.000000
25%           7.000000
50%          16.000000
75%          36.000000
max         109.000000
Name: ARR_DELAY, dtype: float64

In [84]:
delay_reason_in = delayed_inrange[delayed_inrange.columns[-6:-2]].fillna(0)
delay_reason_in

,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
3,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,21.0,7.0,0.0,0.0
9,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,76.0
...,...,...,...,...
573863,10.0,20.0,0.0,11.0
573864,54.0,0.0,0.0,20.0
573865,0.0,0.0,0.0,0.0
573867,0.0,0.0,0.0,0.0


For non-outlier delays, the proportion of NAS delays is much higher while the proportion of weather delays decreased a lot.

In [87]:
fig = px.pie(values = delay_reason_in.sum(), names=delay_reason_in.sum().index, title='Non-outlier Delay Reasons Distribution')
fig.show()

A proportion of 9% outliers may be significant so we analyze the outliers separately as well 

In [28]:
20188/215103

0.0938527124214911

### Delay Time - Conclusions
 - Non-Outliers: $\leq 2h$, mean of 25 min, median of 16 min, std of 25 min
 - Outliers: mean of 216 min, median of 163 min, std of 177 min
 
 38% of all flights flown were delayed. <br>
 9% of delayed flights were outliers, which could be a significant portion and warrants examination.

### Delay Reasons - Conclusions
 - Delays due to a aircraft already late from previous airports is the most common reason for delayed flights. 
 - Weather conditions seem to result in signficantly longer delays since the proportion of weather related reasons for the outliers is more than double the proportion for the non-outliers.
 - NAS delays (from traffic, operations, etc.) is the second most common reason overall, but the proportion of NAS delays in non-outliers is around double the proportion of the outliers.

## Distance of Flight

## Departure Time of Flight

## Week of Day of Flight

## Origin and Destination